In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---
# Zebrafish
## Alemany_Fin

In [ ]:
alemany_fin = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Alemany_Fin/data.h5")
utils.peek(alemany_fin, "build/fin/Alemany_Fin")
alemany_fin.obs.head()

In [ ]:
alemany_fin.obs["ClusterID"] = "cluster_" + alemany_fin.obs["ClusterID"].astype(int).astype(str)
alemany_fin.obs.dtypes

In [ ]:
opt_model_kwargs = dict(batch_effect="donor")
alemany_fin_model = cb.directi.fit_DIRECTi(
    alemany_fin, alemany_fin.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
alemany_fin.latent = alemany_fin_model.inference(alemany_fin)

In [ ]:
ax = alemany_fin.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/fin/Alemany_Fin/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = alemany_fin.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/fin/Alemany_Fin/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = alemany_fin.visualize_latent("ClusterID", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/fin/Alemany_Fin/ClusterID.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = alemany_fin.visualize_latent("donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/fin/Alemany_Fin/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = alemany_fin.visualize_latent("free_annotation", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/fin/Alemany_Fin/free_annotation.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
alemany_fin.write_dataset("build/fin/Alemany_Fin/Alemany_Fin.h5")

In [ ]:
%%capture capio
alemany_fin_models = [alemany_fin_model]
opt_model_kwargs = dict(batch_effect="donor")
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    alemany_fin_models.append(cb.directi.fit_DIRECTi(
        alemany_fin, alemany_fin.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
alemany_fin_blast = cb.blast.BLAST(alemany_fin_models, alemany_fin)
alemany_fin_blast.save("build/fin/Alemany_Fin")

In [ ]:
with open("build/fin/Alemany_Fin/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/fin/Alemany_Fin/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(alemany_fin_blast, "build/fin/Alemany_Fin")

In [ ]:
%%writefile build/fin/Alemany_Fin/predictable.txt
cell_ontology_class
cell_type1
ClusterID